# GPU

查看GPU状态

使用GoogleColab
拿到了一块Tesla T4 15G

In [1]:
# !nvidia-smi

Sat Feb  4 06:12:28 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   62C    P0    29W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# !pip install torch

计算设备

In [2]:
import torch
from torch import nn

torch.device('cpu'), torch.cuda.device('cuda'), torch.cuda.device('cuda:1')

(device(type='cpu'),
 <torch.cuda.device at 0x7fdafec93700>)

查询可用gpu数量

In [3]:
torch.cuda.device_count()

1

函数允许在请求不到GPU的情况下运行代码

In [4]:
def try_gpu(i=0): 
    """如果存在，则返回gpu(i)，否则返回cpu()"""
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')

def try_all_gpus(): 
    """返回所有可用的GPU，如果没有GPU，则返回[cpu(),]"""
    devices = [torch.device(f'cuda:{i}')
             for i in range(torch.cuda.device_count())]
    return devices if devices else [torch.device('cpu')]

try_gpu(), try_gpu(10), try_all_gpus()

(device(type='cuda', index=0),
 device(type='cpu'),
 [device(type='cuda', index=0)])

# 张量与GPU

In [5]:
x = torch.tensor([1, 2, 3])
x.device

device(type='cpu')

在GPU上存储张量

In [6]:
X = torch.ones(2, 3, device=try_gpu())
X

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')

第二块GPU

In [7]:
Y = torch.rand(2, 3, device=try_gpu(1))
Y

tensor([[0.4078, 0.5411, 0.6719],
        [0.6562, 0.8515, 0.6696]])

复制

X+Y

In [10]:
# Z = X.cuda(1)
Z = X.cuda(0)
print(X)
print(Z)

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')
tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')


在同一块GPU上可以运算

In [12]:
X + Z

tensor([[2., 2., 2.],
        [2., 2., 2.]], device='cuda:0')

已经存在于一个GPU上的数据的调用，不会复制并分配新的内存

In [14]:
Z.cuda(0) is Z

True

# 神经网络与GPU

In [15]:
net = nn.Sequential(nn.Linear(3, 1))
net = net.to(device=try_gpu())

In [16]:
net(X)

tensor([[-0.8853],
        [-0.8853]], device='cuda:0', grad_fn=<AddmmBackward0>)

确认模型和参数在一个GPU上

In [17]:
net[0].weight.data.device

device(type='cuda', index=0)